##Clustering

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/datasetSeleccionadasPreClustering.csv', decimal='.')

df_periods = df[['home_team', 'goals_scored_ht_avg_H', 'goals_scored_ft_avg_H', 'goals_conced_ht_avg_H', 'goals_conced_ft_avg_H']]

In [ ]:
#Eliminacion promedios y columnas que no se van a utilizar
eliminacion_clumnas_menor_60 = ['id', 'home_clearances', 'home_fouls_conceded', 'home_offsides', 'home_tackles', 'away_clearances', 'away_fouls_conceded',
                                'away_offsides', 'away_tackles', 'clearances_avg_H', 'fouls_conceded_avg_H', 'offsides_avg_H', 'tackles_avg_H',
                                'fouls_conceded_avg_A', 'offsides_avg_A', 'tackles_avg_A']
df.drop(eliminacion_clumnas_menor_60, axis=1, inplace=True)

In [ ]:
# Total de goles anotados y recibidos en la primera y segunda mitad para cada equipo local
performance = df.groupby('home_team').agg({
    'goals_scored_ht_avg_H': 'sum',
    'goals_scored_ft_avg_H': 'sum',
    'goals_conced_ht_avg_H': 'sum',
    'goals_conced_ft_avg_H': 'sum'
}).reset_index()

# Calcular la diferencia de goles en la primera y segunda mitad
performance['goal_diff_1st_half'] = performance['goals_scored_ht_avg_H'] - performance['goals_conced_ht_avg_H']
performance['goal_diff_2nd_half'] = performance['goals_scored_ft_avg_H'] - performance['goals_conced_ft_avg_H']

# Calcular la diferencia total de goles entre la segunda mitad y la primera mitad
performance['goal_diff_total'] = (performance['goals_scored_ft_avg_H'] - performance['goals_conced_ft_avg_H']) - (performance['goals_scored_ht_avg_H'] - performance['goals_conced_ht_avg_H'])

# Redondear a dos decimales
performance['goal_diff_1st_half'] = performance['goal_diff_1st_half'].round(2)
performance['goal_diff_2nd_half'] = performance['goal_diff_2nd_half'].round(2)
performance['goal_diff_total'] = performance['goal_diff_total'].round(2)

# Columnas relevantes
performance_clustering = performance[['home_team', 'goal_diff_1st_half', 'goal_diff_2nd_half', 'goal_diff_total']]

# Nueva columna para clasificar los equipos según el periodo de mejor rendimiento
conditions = [
    (performance_clustering['goal_diff_total'] <= -2),
    (performance_clustering['goal_diff_total'].abs() < 2),
    (performance_clustering['goal_diff_total'] >= 2)
]
choices = ['Mejor en primera parte', 'Equilibrado', 'Mejor en segunda parte']
performance_clustering['periodo_mejor_rendimiento'] = np.select(conditions, choices, default='')

performance_clustering = performance_clustering.sort_values(by=['goal_diff_total'], ascending=True)

output_path = '/content/drive/MyDrive/TFM/clustered_performance_first_half_vs_second_half_corrected.csv'
performance_clustering.to_csv(output_path, index=False)

performance_clustering.head()

<ipython-input-70-99307fbd2811>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_clustering['periodo_mejor_rendimiento'] = np.select(conditions, choices, default='')


,home_team,goal_diff_1st_half,goal_diff_2nd_half,goal_diff_total,periodo_mejor_rendimiento
15,hull city,1.79,-3.11,-4.90,Mejor en primera parte
24,queens park rangers,-5.72,-9.40,-3.68,Mejor en primera parte
4,blackburn rovers,-0.04,-2.86,-2.82,Mejor en primera parte
33,west bromwich albion,-8.40,-11.18,-2.78,Mejor en primera parte
0,afc bournemouth,-2.46,-4.99,-2.53,Mejor en primera parte
